# Markdown cell (not executable, for documentation)

# Task 4: Insights and Recommendations for Fintech Apps

This notebook orchestrates the analysis for Task 4 of the B5W2 challenge. It:
- Connects to the Oracle database (bank_reviews) running on Docker to fetch review data.
- Performs exploratory data analysis (EDA).
- Uses modular scripts to analyze sentiments, derive insights, and generate visualizations.
- Summarizes drivers, pain points, and recommendations for the final report.

## Imports and Setup

In [1]:
import os
import sys
import logging
import pandas as pd

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import oracledb
from scripts.insights_analysis.insights_analysis import compute_bank_comparison, identify_drivers_pain_points
from scripts.insights_analysis.visualization import plot_sentiment_trends, plot_rating_distribution, plot_keyword_cloud, plot_sentiment_by_rating, plot_theme_distribution
from dotenv import load_dotenv


# Set plot style for consistency
sns.set_theme()
sns.set_palette('husl')

In [3]:
df = pd.read_csv(
    '/home/abel/Desktop/Projects/fintech-review-analytics/data/thematically_analyzed/bank_of_abyssinia_reviews_thematic_analysis.csv')
df.columns

Index(['review_id', 'review_text', 'rating', 'date', 'bank_name', 'source',
       'processed_text', 'identified_theme'],
      dtype='object')

## Connect to Oracle Database and Load Data

In [5]:
load_dotenv()

ORACLE_USER = os.getenv("ORACLE_USER")
ORACLE_PASSWORD = os.getenv("ORACLE_PASSWORD")
ORACLE_DSN = os.getenv("ORACLE_DSN")

# Database connection parameters (adjust as per your Docker setup)
dsn = oracledb.makedsn(host='localhost', port=1521, service_name='XE')
connection = oracledb.connect(
    user=ORACLE_USER, password=ORACLE_PASSWORD, dsn=ORACLE_DSN)

# Query to fetch review data
query = """
SELECT review_id, review_text, rating, review_date, bank_name, source, processed_text, identified_theme
FROM reviews
"""
df = pd.read_sql(query, connection)

# Load data into DataFrame
df = pd.read_sql(query, connection)

# Close connection
connection.close()

# Basic EDA
print('Dataset Info:')
print(df.info())
print('\nSummary Statistics:')
print(df.describe(include='all'))

# Check for missing values
print('\nMissing Values:')
print(df.isnull().sum())
print(df.head())
# Ensure date is in datetime format
df.columns = [col.lower() for col in df.columns]
df['review_date'] = pd.to_datetime(df['review_date'])

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   REVIEW_ID         1200 non-null   object        
 1   REVIEW_TEXT       1200 non-null   object        
 2   RATING            1200 non-null   int64         
 3   REVIEW_DATE       1200 non-null   datetime64[ns]
 4   BANK_NAME         1200 non-null   object        
 5   SOURCE            1200 non-null   object        
 6   PROCESSED_TEXT    1182 non-null   object        
 7   IDENTIFIED_THEME  1200 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 75.1+ KB
None

Summary Statistics:
              REVIEW_ID REVIEW_TEXT       RATING          REVIEW_DATE  \
count              1200        1200  1200.000000                 1200   
unique             1200         809          NaN                  NaN   
top     Dashen_Bank_383  

/tmp/ipykernel_179866/2766714556.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
/tmp/ipykernel_179866/2766714556.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


## Bank Comparison Analysis

In [7]:
# Compute bank comparison (average sentiment and rating)
os.makedirs('reports', exist_ok=True)

# Ensure 'bank' column exists for compatibility with compute_bank_comparison
if 'bank' not in df.columns and 'bank_name' in df.columns:
    df['bank'] = df['bank_name']

bank_comparison = compute_bank_comparison(df)
print('\nBank Comparison (Average Sentiment and Rating):')
print(bank_comparison)
# Save comparison for report
bank_comparison.to_csv('reports/bank_comparison.csv', index=False)

KeyError: "Column(s) ['sentiment_score'] do not exist"

## Drivers and Pain Points

In [ ]:
# Identify drivers and pain points per bank
import json
drivers_pain_points = identify_drivers_pain_points(df)
print('\nDrivers and Pain Points:')
for bank, insights in drivers_pain_points.items():
    print(f'{bank}:')
    print(f"  Drivers: {insights['drivers']}")
    print(f"  Pain Points: {insights['pain_points']}")

# Save insights as JSON for report
with open('reports/drivers_pain_points.json', 'w') as f:
    json.dump(drivers_pain_points, f, indent=2)

KeyError: 'identified_theme'

## Visualizations

In [ ]:
# Generate and save visualizations
plot_sentiment_trends(df, save_path='reports/sentiment_trends.png')
plot_rating_distribution(df, save_path='reports/rating_distribution.png')
plot_keyword_cloud(df, save_path='reports/keyword_cloud.png')
plot_sentiment_by_rating(df, save_path='reports/sentiment_by_rating.png')
plot_theme_distribution(df, save_path='reports/theme_distribution.png')

# Display plots in notebook (optional, for verification)
plt.figure(figsize=(12, 8))
plot_sentiment_trends(df)
plt.show()

plt.figure(figsize=(12, 8))
plot_rating_distribution(df)
plt.show()

plt.figure(figsize=(12, 8))
plot_keyword_cloud(df)
plt.show()

plt.figure(figsize=(12, 8))
plot_sentiment_by_rating(df)
plt.show()

plt.figure(figsize=(12, 8))
plot_theme_distribution(df)
plt.show()

## Recommendations

In [ ]:
# Define recommendations based on insights
recommendations = {
    'Commercial Bank of Ethiopia': [
        'Optimize app loading times during transfers to address slow performance.',
        'Add budgeting tools to enhance user engagement and retention.'
    ],
    'Bank of Abyssinia': [
        'Fix frequent login errors to improve user access and reduce frustration.',
        'Enhance UI navigation for a more intuitive user experience.'
    ],
    'Dashen Bank': [
        'Improve transaction speed for transfers to boost reliability.',
        'Integrate fingerprint login for enhanced security and convenience.'
    ]
}

# Print recommendations
print('\nRecommendations:')
for bank, recs in recommendations.items():
    print(f'{bank}:')
    for rec in recs:
        print(f'  - {rec}')

# Save recommendations as JSON for report
with open('reports/recommendations.json', 'w') as f:
    json.dump(recommendations, f, indent=2)